### setup paths

In [6]:
import os
import subprocess

In [1]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [4]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
}

In [5]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [7]:
for path in paths.values():
  if not os.path.exists(path):
    if os.name == 'posix':
      subprocess.run(['mkdir', '-p', path])
    elif os.name == 'nt':
      subprocess.run(['mkdir', path])


### download tensorflow pretrained models from TF model zoo and install TFOD

In [9]:
# https://www.tensorflow.org/install/source_mac#gpu

In [13]:
%pip install wget

  Using cached wget-3.2-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [14]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
  subprocess.run(['git', 'clone', 'https://github.com/tensorflow/models.git'], cwd=paths['APIMODEL_PATH'])

Cloning into 'models'...
Updating files: 100% (3608/3608), done.


In [ ]:
# Install Tensorflow Object Detection

if os.name=='posix':
  subprocess.run(['apt-get', 'install', 'protobuf-compiler'])
  subprocess.run(['cd', 'Tensorflow/models/research'])
  subprocess.run(['protoc', 'object_detection/protos/*.proto', '--python_out=.'])
  subprocess.run(['cp', 'object_detection/packages/tf2/setup.py', './'])
# elif os.name=='nt':
  # for windows download and install protocol buffers